In [1]:
import pandas as pd
import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
merged_data = pd.read_csv('merged_data.csv')
merged_data

,id,created_at,rating,title,content,thank_count,customer_name,customer_id,product_id,year,...,price,original_price,discount_rate,quantity_sold,rating_average,review_count,seller_id,category,seller_name,author_name
0,12559756,2021-10-11 14:01:12,5,Cực kì hài lòng,"Có những người bước đến, họ lấp đầy hạnh phúc ...",389,Vân Anh,22051463,74021317,2021,...,68040,108000,37,70416,5.0,13151,1,tiểu thuyết,Tiki Trading,José Mauro De Vasconcelos
1,16979365,2022-07-07 15:01:37,5,Cực kì hài lòng,"Thấy nhiều bạn chê tiki gói hàng quá, may sao ...",14,Phan Thảo Yến Nhím,27791831,74021317,2022,...,68040,108000,37,70416,5.0,13151,1,tiểu thuyết,Tiki Trading,José Mauro De Vasconcelos
2,14069617,2021-12-25 14:33:51,5,Cực kì hài lòng,"Bìa cực xinh, tiki giao hàng nhanh, sách không...",26,Trần Thị Trang,17748750,74021317,2021,...,68040,108000,37,70416,5.0,13151,1,tiểu thuyết,Tiki Trading,José Mauro De Vasconcelos
3,12322259,2021-09-25 11:17:51,5,Cực kì hài lòng,"Nội dung của sách thì không phải bàn đến rồi, ...",16,Khánh Ly,10149686,74021317,2021,...,68040,108000,37,70416,5.0,13151,1,tiểu thuyết,Tiki Trading,José Mauro De Vasconcelos
4,18368714,2022-12-07 18:48:33,5,Cực kì hài lòng,Một cuốn sách rất đáng đọc về tình yêu thương ...,1,Phương Linh,28545286,74021317,2022,...,68040,108000,37,70416,5.0,13151,1,tiểu thuyết,Tiki Trading,José Mauro De Vasconcelos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590478,8252336,2021-03-08 03:31:15,4,Hài lòng,khâu bảo quản mép cuốn truyện khá kém nên được...,0,Nguyn Trung Kien,15986076,75996803,2021,...,42400,55000,23,409,5.0,57,1,văn học thiếu nhi,Tiki Trading,Roald Dahl
590479,17988874,2022-10-18 06:53:03,4,Hài lòng,Ship nhanh + Bìa bóng đẹp + mực in đậm \r\n*Hi...,0,Lợn Bi Bi,28039159,75996803,2022,...,42400,55000,23,409,5.0,57,1,văn học thiếu nhi,Tiki Trading,Roald Dahl
590480,17848110,2022-10-02 02:13:13,4,Hài lòng,No comment,0,Phạm Thanh Bình,27744978,75996803,2022,...,42400,55000,23,409,5.0,57,1,văn học thiếu nhi,Tiki Trading,Roald Dahl
590481,10193825,2021-06-06 00:50:16,4,Hài lòng,No comment,0,Ha Vuong,5056432,75996803,2021,...,42400,55000,23,409,5.0,57,1,văn học thiếu nhi,Tiki Trading,Roald Dahl


**Recency**

In [16]:
today = datetime.datetime.now()
df_RFM = merged_data.groupby('customer_id').created_at.max().reset_index()
df_RFM.created_at = pd.to_datetime(df_RFM.created_at)
df_RFM['Recency'] = (today - df_RFM['created_at']).dt.days

**frequency**

In [17]:
df_frequency = merged_data.groupby('customer_id').created_at.count().reset_index()
df_frequency.columns = ['customer_id', 'Frequency']
df_RFM = pd.merge(df_frequency, df_RFM, on = 'customer_id')

**Monetary**

In [18]:
df_monetary = merged_data.groupby('customer_id').price.sum().reset_index()
df_monetary.columns = ['customer_id', 'Monetary']
df_RFM = pd.merge(df_monetary, df_RFM, on = 'customer_id')

**Dùng Kmean để chia cụm từng khúc khách hàng.**

In [19]:
# sse = {}
# recency = df_RFM[['Recency']]
# for k in range(1, 10):
#     kmeans = KMeans(n_clusters = k, max_iter= 1000).fit(recency)
#     recency['clusters'] = kmeans.labels_
#     sse[k] = kmeans.inertia_
# plt.plot(list(sse.keys()), list(sse.values()))
# plt.xlabel("ok")
# plt.show()

- Recency

In [20]:
kmeans = KMeans(n_clusters = 5)
kmeans.fit(df_RFM[['Recency']])
df_RFM['RecencyCluster'] = kmeans.predict(df_RFM[['Recency']])

df_new = df_RFM.groupby('RecencyCluster')['Recency'].mean().reset_index()
df_new = df_new.sort_values(by = 'Recency', ascending=False).reset_index(drop = True)
df_new['index'] = df_new.index
df_RFM = pd.merge(df_RFM, df_new[['index', 'RecencyCluster']], on = 'RecencyCluster')
df_RFM = df_RFM.drop(['RecencyCluster'], axis = 1)
df_RFM = df_RFM.rename(columns={'index':'RecencyCluster'})

c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the war

In [21]:
df_new

,RecencyCluster,Recency,index
0,4,2580.746381,0
1,3,1294.663826,1
2,1,971.545214,2
3,0,662.870987,3
4,2,281.537072,4


In [22]:
df_RFM

,customer_id,Monetary,Frequency,created_at,Recency,RecencyCluster
0,22,663000,1,2022-04-03 11:22:32,633,3
1,120,413000,1,2022-03-19 13:42:05,648,3
2,187,152100,1,2022-02-06 16:37:25,688,3
3,247,923140,7,2022-06-21 22:49:34,553,3
4,330,149460,1,2021-12-15 15:27:09,741,3
...,...,...,...,...,...,...
311654,7733750,125000,1,2018-08-26 05:30:25,1949,0
311655,7769221,150000,1,2018-08-24 07:49:03,1951,0
311656,7852188,150000,1,2018-08-28 06:13:17,1947,0
311657,7855270,154000,1,2018-09-04 01:04:55,1940,0


- Frequency

In [23]:
kmeans = KMeans(n_clusters = 5)
kmeans.fit(df_RFM[['Frequency']])
df_RFM['FrequencyCluster'] = kmeans.predict(df_RFM[['Frequency']])

c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the war

In [24]:
df_new = df_RFM.groupby('FrequencyCluster')['Frequency'].mean().reset_index()
df_new = df_new.sort_values(by = 'Frequency').reset_index(drop = True)
df_new['index'] = df_new.index
df_RFM = pd.merge(df_RFM, df_new[['index', 'FrequencyCluster']], on = 'FrequencyCluster')
df_RFM = df_RFM.drop(['FrequencyCluster'], axis = 1)
df_RFM = df_RFM.rename(columns={'index':'FrequencyCluster'})

In [25]:
df_new

,FrequencyCluster,Frequency,index
0,0,1.220984,0
1,3,3.800234,1
2,1,8.746326,2
3,4,18.272313,3
4,2,44.852632,4


In [26]:
df_RFM


,customer_id,Monetary,Frequency,created_at,Recency,RecencyCluster,FrequencyCluster
0,22,663000,1,2022-04-03 11:22:32,633,3,0
1,120,413000,1,2022-03-19 13:42:05,648,3,0
2,187,152100,1,2022-02-06 16:37:25,688,3,0
3,330,149460,1,2021-12-15 15:27:09,741,3,0
4,350,309000,1,2022-02-02 02:41:41,693,3,0
...,...,...,...,...,...,...,...
311654,28059010,4045180,42,2022-12-16 09:53:17,376,4,4
311655,28391752,5891970,57,2023-11-11 00:23:15,46,4,4
311656,28420073,8983430,94,2023-03-17 10:48:12,285,4,4
311657,28435491,2954200,38,2023-11-13 08:37:13,44,4,4


- Monetary

In [27]:
kmeans = KMeans(n_clusters = 5)
kmeans.fit(df_RFM[['Monetary']])
df_RFM['MonetaryCluster'] = kmeans.predict(df_RFM[['Monetary']])

c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the war

c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [28]:
df_new = df_RFM.groupby('MonetaryCluster')['Monetary'].mean().reset_index()
df_new = df_new.sort_values(by = 'Monetary').reset_index(drop = True)
df_new['index'] = df_new.index
df_RFM = pd.merge(df_RFM, df_new[['index', 'MonetaryCluster']], on = 'MonetaryCluster')
df_RFM = df_RFM.drop(['MonetaryCluster'], axis = 1)
df_RFM = df_RFM.rename(columns={'index':'MonetaryCluster'})

In [29]:
df_new

,MonetaryCluster,Monetary,index
0,1,1.139271e+05,0
1,3,3.409675e+05,1
2,0,7.442894e+05,2
3,4,1.617711e+06,3
4,2,4.419622e+06,4


In [30]:
df_RFM

,customer_id,Monetary,Frequency,created_at,Recency,RecencyCluster,FrequencyCluster,MonetaryCluster
0,22,663000,1,2022-04-03 11:22:32,633,3,0,2
1,3904,597000,2,2021-12-22 01:55:21,735,3,0,2
2,5246,860000,2,2022-04-09 13:08:05,627,3,0,2
3,7166,725900,2,2022-07-08 12:16:07,537,3,0,2
4,7552,750000,2,2022-05-21 04:08:14,585,3,0,2
...,...,...,...,...,...,...,...,...
311654,26835881,5576890,71,2023-06-23 04:54:00,187,4,4,4
311655,28033609,3107530,36,2023-05-24 02:17:03,217,4,4,4
311656,28059010,4045180,42,2022-12-16 09:53:17,376,4,4,4
311657,28391752,5891970,57,2023-11-11 00:23:15,46,4,4,4
